# Proyecto Webscraping
Proyecto para la Licenciatura en Datos y Negocios de la Universidad Católica del Uruguay

In [7]:
import requests
from bs4 import BeautifulSoup
import json

## Agregar el URL de la inmobiliaria Infocasas

In [8]:
url = 'https://www.infocasas.com.uy/alquiler/apartamentos'
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

soup

<!DOCTYPE html>
<html class="infocasas-theme" id="bodyTag" lang="es-UY"><head><link as="font" crossorigin="anonymous" href="https://cdn4.fincaraiz.com.co/web/5fdcaa52b06cf_infocdn__infocasas-regular.woff2" rel="preload" type="font/woff"/><link as="font" crossorigin="anonymous" href="https://cdn4.fincaraiz.com.co/web/5fdcaa52af7c5_infocdn__infocasas-bold.woff2" rel="preload" type="font/woff"/><script>
								function getWidth() {
								return Math.max(
									document.body.scrollWidth,
									document.documentElement.scrollWidth,
									document.body.offsetWidth,
									document.documentElement.offsetWidth,
									document.documentElement.clientWidth
								);
								}
								
								function setHeightPushdown(expanded) {
								const width = getWidth();
								const expandible = document.getElementById('expandible');
								if (expandible) {
									const expandedHeight = width < 600 ? '250px' : '250px';
									const collapsedHeight = width < 600 ? '50px' : '90px

### Añadir a la URL más páginas, para así obtener más propiedades

In [69]:
base_url = 'https://www.infocasas.com.uy/alquiler/apartamentos'
for page in range (1, 20):
    # https://www.infocasas.com.uy/alquiler/apartamentos/pagina2
    url = f'{base_url}/pagina{page}'
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')

soup

<!DOCTYPE html>
<html class="infocasas-theme" id="bodyTag" lang="es-UY"><head><link as="font" crossorigin="anonymous" href="https://cdn4.fincaraiz.com.co/web/5fdcaa52b06cf_infocdn__infocasas-regular.woff2" rel="preload" type="font/woff"/><link as="font" crossorigin="anonymous" href="https://cdn4.fincaraiz.com.co/web/5fdcaa52af7c5_infocdn__infocasas-bold.woff2" rel="preload" type="font/woff"/><script>
								function getWidth() {
								return Math.max(
									document.body.scrollWidth,
									document.documentElement.scrollWidth,
									document.body.offsetWidth,
									document.documentElement.offsetWidth,
									document.documentElement.clientWidth
								);
								}
								
								function setHeightPushdown(expanded) {
								const width = getWidth();
								const expandible = document.getElementById('expandible');
								if (expandible) {
									const expandedHeight = width < 600 ? '250px' : '250px';
									const collapsedHeight = width < 600 ? '50px' : '90px

#### Encontrar los Links de cada propiedad

In [70]:
links = soup.find_all('a', class_= 'lc-data')
for link in links:
    print(link.get('href'))


/alquilamos-amplio-y-luminoso-apto-de-2-dormitorios-con-balcon-en-cordon/192844459
/estrene-apartamento-un-dormitorio-cordon-sur/192866856
/alquiler-amplio-apto-reciclado-a-nuevo-en-centro/192852720
/funcional-apartamento-en-alquiler-con-1-dormitorio-y-ventilacion-natural/192905374
/alquiler-3-dormitorios-y-garage-en-pocitos/192547844
/alquiler-apartamento-1-dormitorio-en-aguada-cochera-opcional/192748364
/funcional-apartamento-en-alquiler-con-pisos-de-madera-por-19500/192902766
/apartamento-al-frente-pocitos/192903308
/hermosa-planta-con-excelente-luz-y-ubicacion/192397664
/apartamento-en-alquiler-de-1-dormitorio-en-villa-biarritz/192026025
/apartamento-en-alquiler-monoambiente/192368261
/a-1405alquiler-apartamento-3-dorm-1-baño-c-garage-en-buceo/192742302
/apartamento-en-haras-del-lago-de-1-dormitorios-con-balcon-y-garage/192849255
/alquiler-de-apartamento-de-1-dormitorio-a-estrenar-en-cordon/192581695
/departamento-de-3-dormitorios-con-vista-al-mar/192693197
/la-blanqueada-ref-e1788

#### Unimos el 'href' con la URL para que nos de el link completo

In [71]:
links_soup = soup.find_all('a', class_= 'lc-data')
all_links = []

for l in links_soup:
    link = base_url + l['href']
    all_links.append(link)

all_links

['https://www.infocasas.com.uy/alquiler/apartamentos/alquilamos-amplio-y-luminoso-apto-de-2-dormitorios-con-balcon-en-cordon/192844459',
 'https://www.infocasas.com.uy/alquiler/apartamentos/estrene-apartamento-un-dormitorio-cordon-sur/192866856',
 'https://www.infocasas.com.uy/alquiler/apartamentos/alquiler-amplio-apto-reciclado-a-nuevo-en-centro/192852720',
 'https://www.infocasas.com.uy/alquiler/apartamentos/funcional-apartamento-en-alquiler-con-1-dormitorio-y-ventilacion-natural/192905374',
 'https://www.infocasas.com.uy/alquiler/apartamentos/alquiler-3-dormitorios-y-garage-en-pocitos/192547844',
 'https://www.infocasas.com.uy/alquiler/apartamentos/alquiler-apartamento-1-dormitorio-en-aguada-cochera-opcional/192748364',
 'https://www.infocasas.com.uy/alquiler/apartamentos/funcional-apartamento-en-alquiler-con-pisos-de-madera-por-19500/192902766',
 'https://www.infocasas.com.uy/alquiler/apartamentos/apartamento-al-frente-pocitos/192903308',
 'https://www.infocasas.com.uy/alquiler/apa

#### Encontrar la Ciudad en la cuál se encuentra la propiedad

In [72]:
locations = soup.find_all('strong', class_="lc-location body body-1 body-bold medium")

foo = [location.text.strip() for location in locations]
cities = [city.split(',')[1] for city in foo]
cities

[' Montevideo',
 ' Montevideo',
 ' Montevideo',
 ' Montevideo',
 ' Montevideo',
 ' Montevideo',
 ' Montevideo',
 ' Montevideo',
 ' Montevideo',
 ' Montevideo',
 ' Montevideo',
 ' Montevideo',
 ' Canelones',
 ' Montevideo',
 ' Maldonado',
 ' Montevideo',
 ' Canelones',
 ' Montevideo',
 ' Montevideo',
 ' Montevideo',
 ' Montevideo']

#### Encontrar la cantidad de Habitaciones por propiedad

In [74]:
rooms = soup.find_all('strong')
dorms = []
for r in rooms:
    texto = r.text.strip()
    if "Dorm" in texto or "Dorms" in texto:
        dorms.append(texto.strip().split()[0])

dorms


['2',
 '1',
 '1',
 '1',
 '3',
 '1',
 '1',
 '2',
 '1',
 '3',
 '1',
 '1',
 '3',
 '1',
 '2',
 '2',
 '3',
 '3',
 '1']

#### Encontrar el Precio de cada propiedad

In [75]:
price_soup = soup.find_all('p', class_='main-price')
prices = []
for price in price_soup:
    prices.append(price.text.strip().split()[1])

prices


['33.000',
 '24.500',
 '21.800',
 '21.000',
 '49.500',
 '25.000',
 '19.500',
 '21.000',
 '35.000',
 '30.000',
 '19.000',
 '32.500',
 '33.000',
 '28.000',
 '1.600',
 '24.500',
 '2.700',
 '2.100',
 '1.800',
 '25.000',
 '22.000']

#### La consigna pide los precios en dolares, por lo que lo transformamos

In [76]:
price_soup = soup.find_all('p', class_='main-price')
tipo_cambio = 39.88
prices = []
for price in price_soup:
    texto = price.text.strip()
    partes = texto.split()
    simbolo = partes[0]
    valor = partes[1]
    if simbolo == '$':
        valor_numerico = float(valor.replace('.', '').replace(',', '.'))
        valor_final = valor_numerico * tipo_cambio
        valor = valor_final
    prices.append(valor)

prices

[1316040.0,
 977060.0000000001,
 869384.0,
 837480.0,
 1974060.0000000002,
 997000.0000000001,
 777660.0,
 837480.0,
 1395800.0,
 1196400.0,
 757720.0,
 1296100.0,
 1316040.0,
 1116640.0,
 '1.600',
 977060.0000000001,
 '2.700',
 '2.100',
 '1.800',
 997000.0000000001,
 877360.0]

#### Encontra el tamaño en m²

In [77]:
tamanos_soup = soup.find_all('strong')
tamanos = []
for tamano in tamanos_soup:
    texto = tamano.text.strip()
    if "m²" in texto:
        tamanos.append(texto.strip().split()[0])

tamanos

['82',
 '48',
 '47',
 '40',
 '95',
 '58',
 '35',
 '30',
 '83',
 '60',
 '29',
 '75',
 '60',
 '56',
 '41',
 '149',
 '101',
 '150',
 '49',
 '60']

In [ ]:
# Aqui voy a tener un monton de listas del mismo tamaño
# Crear una lista vacia para guardar todos los diccionarios
# Hago un for de 0 hasta el len()
# Dentro del for tengo que generar un diccionario, asignandole a cada clave el valor que tengo en cada lista


In [ ]:
# Recorrer esa nueva lista
# Preguntar si la ciudad ya existe en mi lista, si no existe, crear la lista para esa ciudad, si no existe, hacer append de esa ciudad.

# Cuando termino esto, tengo tantas listas como ciudades
# Ahora puedo crear el formato que pidieron

In [78]:
propiedades = []
for ciudad, precio, dormitorios, tamano, link in zip(cities, prices, dorms, tamanos, all_links):
    propiedad = {
        "precio": precio,
        "habitaciones": dormitorios,
        "tamano": tamano,
        "link": link
    }
    propiedades.append(propiedad)

propiedades


[{'precio': 1316040.0,
  'habitaciones': '2',
  'tamano': '82',
  'link': 'https://www.infocasas.com.uy/alquiler/apartamentos/alquilamos-amplio-y-luminoso-apto-de-2-dormitorios-con-balcon-en-cordon/192844459'},
 {'precio': 977060.0000000001,
  'habitaciones': '1',
  'tamano': '48',
  'link': 'https://www.infocasas.com.uy/alquiler/apartamentos/estrene-apartamento-un-dormitorio-cordon-sur/192866856'},
 {'precio': 869384.0,
  'habitaciones': '1',
  'tamano': '47',
  'link': 'https://www.infocasas.com.uy/alquiler/apartamentos/alquiler-amplio-apto-reciclado-a-nuevo-en-centro/192852720'},
 {'precio': 837480.0,
  'habitaciones': '1',
  'tamano': '40',
  'link': 'https://www.infocasas.com.uy/alquiler/apartamentos/funcional-apartamento-en-alquiler-con-1-dormitorio-y-ventilacion-natural/192905374'},
 {'precio': 1974060.0000000002,
  'habitaciones': '3',
  'tamano': '95',
  'link': 'https://www.infocasas.com.uy/alquiler/apartamentos/alquiler-3-dormitorios-y-garage-en-pocitos/192547844'},
 {'preci

In [79]:
ciudades_dict= {}
propiedades = []
for ciudad, precio, dormitorios, tamano, link in zip(cities, prices, dorms, tamanos, all_links):
    propiedad = {
        "precio": precio,
        "habitaciones": dormitorios,
        "tamano": tamano,
        "link": link
    }

    if ciudad not in ciudades_dict:
         ciudades_dict[ciudad] = []
    ciudades_dict[ciudad].append(propiedad)

for ciudad in ciudades_dict:
    ciudades_dict[ciudad] = ciudades_dict[ciudad][:10]

ciudades_dict

{' Montevideo': [{'precio': 1316040.0,
   'habitaciones': '2',
   'tamano': '82',
   'link': 'https://www.infocasas.com.uy/alquiler/apartamentos/alquilamos-amplio-y-luminoso-apto-de-2-dormitorios-con-balcon-en-cordon/192844459'},
  {'precio': 977060.0000000001,
   'habitaciones': '1',
   'tamano': '48',
   'link': 'https://www.infocasas.com.uy/alquiler/apartamentos/estrene-apartamento-un-dormitorio-cordon-sur/192866856'},
  {'precio': 869384.0,
   'habitaciones': '1',
   'tamano': '47',
   'link': 'https://www.infocasas.com.uy/alquiler/apartamentos/alquiler-amplio-apto-reciclado-a-nuevo-en-centro/192852720'},
  {'precio': 837480.0,
   'habitaciones': '1',
   'tamano': '40',
   'link': 'https://www.infocasas.com.uy/alquiler/apartamentos/funcional-apartamento-en-alquiler-con-1-dormitorio-y-ventilacion-natural/192905374'},
  {'precio': 1974060.0000000002,
   'habitaciones': '3',
   'tamano': '95',
   'link': 'https://www.infocasas.com.uy/alquiler/apartamentos/alquiler-3-dormitorios-y-gara

In [80]:
resultado = {
    'ciudades': [
        {
            'nombre': ciudad,
            'propiedades': propiedades
        }
        for ciudad, propiedades in ciudades_dict.items()
    ]
}

In [81]:
with open('ciudades_dict.json', 'w') as outfile:
    json.dump(resultado, outfile, indent=4)


#### Fórmula para saber la cantidad de ciudades
Ya que por ejemplo para Canelones o Maldonado, son menos de 10 propiedades

In [82]:
from collections import Counter

conteo_ciudades = Counter(cities)
print(conteo_ciudades)


Counter({' Montevideo': 18, ' Canelones': 2, ' Maldonado': 1})
